In [1]:
import numpy as np
import torch
from torch import nn
import nltk
nltk.download('stopwords')
import pandas as pd
from nltk.corpus import stopwords
import torch.optim as optim
from tqdm import tqdm
import os
import pandas as pd
import tensorflow as tf
import torch
!pip install transformers
from transformers import BertTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 552kB 3.4MB/s 
     |████████████████████████████████| 3.7MB 15.9MB/s 
     |████████████████████████████████| 870kB 28.2MB/s 
     |████████████████████████████████| 1.0MB 41.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=ebe6c941665276697b32e28b9ee1244ef174ce18b7d6c191d4e0b52c1059395e
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import BertModel
bert_model=BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data preparation

In [0]:
training_data=[]
with open('/content/drive/My Drive/train.txt','r') as f:
  line=f.readline()
  while(line):
    line=f.readline()
    if(line.strip()==''):
       training_data.append({'sent':'','label':''})
    else: 
      line=line.split()  
      training_data[-1]['sent']+=' '+str(line[0])
      training_data[-1]['label']+=' '+str(line[-1])

validation_data=[]
with open('/content/drive/My Drive/valid.txt','r') as f:
  line=f.readline()
  while(line):
    line=f.readline()
    if(line.strip()==''):
       validation_data.append({'sent':'','label':''})
    else: 
      line=line.split()  
      validation_data[-1]['sent']+=' '+str(line[0])
      validation_data[-1]['label']+=' '+str(line[-1])   

In [0]:
df=pd.DataFrame(training_data)
df_valid=pd.DataFrame(validation_data)

In [0]:
df['label'].replace('',np.nan,inplace=True)
df_valid['label'].replace('',np.nan,inplace=True)

In [6]:
print(df.isna().sum())
print(df_valid.isna().sum())

sent     0
label    2
dtype: int64
sent     0
label    2
dtype: int64


In [0]:
df.dropna(inplace=True)

In [8]:
df.iloc[0]['sent']

' EU rejects German call to boycott British lamb .'

In [0]:
def convert_to_ints(sent):
    tokens=tokenizer.tokenize(sent)
    tokens=tokenizer.convert_tokens_to_ids(tokens)     
    return tokens

In [0]:
df['encoding']=df['sent'].apply(lambda x:convert_to_ints(x))
df['attn_mask']=df['encoding'].apply(lambda x:[0 if tok==tokenizer.convert_tokens_to_ids('[PAD]') else 1 for tok in x])
df_valid['encoding']=df_valid['sent'].apply(lambda x:convert_to_ints(x))
df_valid['attn_mask']=df_valid['encoding'].apply(lambda x:[0 if tok==tokenizer.convert_tokens_to_ids('[PAD]') else 1 for tok in x])

In [0]:
unique_labels=np.unique(''.join(list(df['label'])).split())
tag_to_index={}
index_to_tag={}
for i,lbl in enumerate(list(unique_labels)):
  tag_to_index[lbl]=i
  index_to_tag[i]=lbl

In [0]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))    

In [0]:
df['label_tokenized']=df['label'].apply(lambda x: x.split())

In [0]:
for index,row in df.iterrows():
  tokenized=tokenizer.tokenize(row['sent'])
  if(len(row['encoding'])!=len(row['label_tokenized']) and sum([1 if(tok.startswith('##')) else 0 for tok in tokenized])>0):
    for i in range(len(tokenized)):
      if(tokenized[i].startswith('##')):
        df.iloc[index]['label_tokenized'].insert(i,'O')
  

In [0]:
df_valid.dropna(inplace=True)
df_valid['label_tokenized']=df_valid['label'].apply(lambda x: x.split())
for index,row in df_valid.iterrows():
  tokenized=tokenizer.tokenize(row['sent'])
  if(len(row['encoding'])!=len(row['label_tokenized']) and sum([1 if(tok.startswith('##')) else 0 for tok in tokenized])>0):
    for i in range(len(tokenized)):
      if(tokenized[i].startswith('##')):
        df.iloc[index]['label_tokenized'].insert(i,'O')

In [0]:
mask=[]
for index,row in df.iterrows():
  mask.append(len(row['encoding'])==len(row['label_tokenized']))
df=df[mask] 

mask=[]
for index,row in df_valid.iterrows():
  mask.append(len(row['encoding'])==len(row['label_tokenized']))
df_valid=df_valid[mask]

# Model

In [0]:
class BERT_CRF(nn.Module):
    def __init__(self,hidden_size,tag_to_index):
        super(BERT_CRF, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        for p in self.bert_layer.parameters():
            p.requires_grad = False

        self.target_size=len(tag_to_index)
        # self.embed = nn.Embedding(vocab_size, embedding_size)
        # self.lstm = nn.LSTM(embedding_size, hidden_size // 2,num_layers=2, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_size, self.target_size)
        self.transitions = nn.Parameter(torch.randn(self.target_size, self.target_size))
        self.start_transition=nn.Parameter(torch.randn(1, self.target_size))
        self.end_transition=nn.Parameter(torch.randn(1, self.target_size))
        # self.hidden=self.init_hidden(hidden_size)
    def _get_lstm_features(self,sentence,attn_mask):
        cont_reps, _ = self.bert_layer(sentence, attention_mask = attn_mask)
        output=self.hidden2tag(cont_reps)
        return output
    def _forward(self,features):   
        temp= self.start_transition.data+features[0,0,:]
        labels=[]
        lbl=torch.argmax(temp,1)
        # score=torch.max(veterbi_var)
        labels.append(lbl)
        # labels.append(argmax(veterbi_var))
        # score=torch.max(veterbi_var)        
        for indx in range(1,features.shape[1]):
            veterbi_var=[]
            for tag in range(features.shape[2]):                             
                veterbi_var.append(log_sum_exp(temp+self.transitions[:,tag].view(1,9)+features[0,indx,tag].expand(1,9)).view(1,1))
            # print(veterbi_var)                   
            temp=torch.cat(veterbi_var,dim=1)
            # print(temp.shape)
            lbl=torch.argmax(temp,1)
                # score=torch.max(veterbi_var)
            labels.append(lbl)
            # score=torch.max(veterbi_var)   
        temp+=self.end_transition.view(1,-1)
        # print(torch.min(temp))
        score=log_sum_exp(temp)
        return score,labels
    def _score_sentence(self,feats,tags):
        score=feats[0,0,tags[0]]+self.start_transition[0,tags[0]]
        for indx in range(1,feats.shape[1]):
            # print(feats.shape,tags[indx],indx,tags.shape)
            score+=feats[0,indx,tags[indx]]+self.transitions[tags[indx-1],tags[indx]]
        score+=self.end_transition[0,tags[-1]] 
        return score
    def _neg_log_likelihood(self,sentence,targets,attn_mask):
        feats = self._get_lstm_features(sentence,attn_mask)
        forward_score,_=self._forward(feats)        
        gold_score = self._score_sentence(feats, targets.view(-1)) 
        # print('forward score {}'.format(forward_score),'gold score {}'.format(gold_score)) 
        loss=forward_score - gold_score     
        return loss
    def _viterbi(self,features):        
        temp= self.start_transition.data.view(1,-1)+features[0,0,:].view(1,-1)
        labels=[]
        # print(temp)
        temp,lbl=torch.max(temp,1)
        # print(temp,lbl)
        labels.append(lbl)     
        for indx in range(1,features.shape[1]):
            veterbi_var=[]
            for tag in range(features.shape[2]):                            
                veterbi_var.append(temp.view(1,1)+self.transitions[lbl[-1],tag].view(1)+features[0,indx,tag].view(1))
            # print(veterbi_var)                   
            temp=torch.cat(veterbi_var,dim=1)
            # lbl=torch.argmax(temp,1)
            temp,lbl=torch.max(temp,1)
            labels.append(lbl)  
        temp=temp.view(1,1)+self.end_transition[0,lbl].view(1,1)
        return temp,labels  
    def _predict(self,sentence,attn_mask):
        feats = self._get_lstm_features(sentence,attn_mask)
        score,labels=self._viterbi(feats)
        return labels

# Training

In [0]:
import pickle
from torch.nn.utils import clip_grad_norm_
# vocab_size=len(vocab_to_int)
hidden_size=768
# embedding_size=50
df_valid.dropna(inplace=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model=BERT_CRF(hidden_size,tag_to_index).to(device)
# if(os.path.exists('/content/drive/My Drive/model.pt')):
#   model.load_state_dict(torch.load('/content/drive/My Drive/model.pt'))
optimizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-4)
losses=[]
# if(os.path.exists('/content/drive/My Drive/losses.pkl')):
#   with open('/content/drive/My Drive/losses.pkl','rb') as f:
#     losses=pickle.load(f)
for epoch in tqdm(range(100)): 
    for index,row in df.iterrows():
        model.zero_grad()
        sent=torch.tensor(row['encoding'],dtype=torch.long).view(1,-1).to(device)
        targets = torch.tensor([tag_to_index[t] for t in row['label_tokenized']], dtype=torch.long).view(1,-1).to(device)
        attn_mask=torch.tensor(row['attn_mask'],dtype=torch.long).view(1,-1).to(device)
        loss = model._neg_log_likelihood(sent, targets,attn_mask)
        loss.backward()
        # print(loss.data.item())
        clip_grad_norm_(model.parameters(), 1)
        optimizer.step() 
        losses.append(loss)
    val_loss=0    
    for index,row in df_valid.iterrows():
        model.zero_grad()
        sent=torch.tensor(row['encoding'],dtype=torch.long).view(1,-1).to(device)
        targets = torch.tensor([tag_to_index[t] for t in row['label_tokenized']], dtype=torch.long).view(1,-1).to(device)
        attn_mask=torch.tensor(row['attn_mask'],dtype=torch.long).view(1,-1).to(device)
        val_loss+= model._neg_log_likelihood(sent, targets,attn_mask).data.item()    
    val_loss=val_loss/len(df_valid)
    print('training loss:{}'.format(loss),'validation loss:{}'.format(val_loss))
    if(len(losses)>0 and val_loss<min(losses)):
      torch.save(model.state_dict(), '/content/drive/My Drive/model.pt')            
    losses.append(val_loss)

# Testing

In [0]:
def process_input(file):
    valid_data=[]
    with open(file,'r') as f:
      line=f.readline()
      while(line):
        line=f.readline()
        if(line.strip()==''):
          valid_data.append({'sent':'','label':''})
        else: 
          line=line.split()  
          valid_data[-1]['sent']+=' '+str(line[0])
          valid_data[-1]['label']+=' '+str(line[-1])
    df=pd.DataFrame(valid_data)
    df['label'].replace('',np.nan,inplace=True)
    df.dropna(inplace=True)
    df['encoding']=df['sent'].apply(lambda x:convert_to_ints(x))
    target=[]
    return df    

In [19]:
hidden_size=768
df_valid.dropna(inplace=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model=BERT_CRF(hidden_size,tag_to_index).to(device)
if(os.path.exists('/content/drive/My Drive/model_BERT.pt')):
  model.load_state_dict(torch.load('/content/drive/My Drive/model_BERT.pt'))
i=0
for index,row in df_valid.iterrows():
    # print(row['sent'])
    sent=torch.tensor(row['encoding'],dtype=torch.long).view(1,-1).to(device)   
    attn_mask=torch.tensor(row['attn_mask'],dtype=torch.long).view(1,-1).to(device)
    prediction=model._predict(sent,attn_mask)
    print(row['sent'])
    print(row['label_tokenized'])
    # print(row['encoding'])
    print([index_to_tag[int(t)] for t in prediction])
    # print(row['label'])
    i+=1
    if(i%10==0):
      break

 CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
 Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .
['O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['B-PER', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
 At the Oval , Surrey captain Chris Lewis , another man dumped by England , continued to silence his critics as he followed his four for 45 on Thursday with 80 not out on Friday in the match against Warwickshire .
['O', 'O', 'B-LOC', 'O', 'B-ORG', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

# Metrics

In [27]:
from sklearn.metrics import multilabel_confusion_matrix,classification_report
y_true=[]
y_pred=[]
for index,row in df_valid.iterrows():
    sent=torch.tensor(row['encoding'],dtype=torch.long).view(1,-1).to(device)   
    attn_mask=torch.tensor(row['attn_mask'],dtype=torch.long).view(1,-1).to(device)
    prediction=model._predict(sent,attn_mask)
    y_true.append(row['label_tokenized'])
    y_pred.append([index_to_tag[int(t)] for t in prediction])
classification_report([y for t in y_true for y in t ],[y for t in y_pred for y in t ])


ERROR! Session/line number was not unique in database. History logging moved to new session 59


'              precision    recall  f1-score   support\n\n       B-LOC       0.68      0.67      0.68       198\n      B-MISC       0.63      0.33      0.43       151\n       B-ORG       0.78      0.68      0.72       311\n       B-PER       0.59      0.55      0.57        80\n       I-LOC       0.00      0.00      0.00        35\n      I-MISC       1.00      0.05      0.09        85\n       I-ORG       0.68      0.16      0.26        94\n       I-PER       1.00      0.03      0.06        34\n           O       0.95      1.00      0.97      6727\n\n    accuracy                           0.93      7715\n   macro avg       0.70      0.38      0.42      7715\nweighted avg       0.92      0.93      0.91      7715\n'

# Validation Losses

In [20]:
import pickle
with open('/content/drive/My Drive/losses_BERT.pkl','rb') as f:
  losses=pickle.load(f)
losses  

[4.284537094457446,
 4.315339248871135,
 4.364466776675949,
 4.428749782857653,
 4.506361849317563,
 4.595588369585962,
 4.695175827902372,
 4.804076100541689,
 4.921474426228786,
 5.046526728071103,
 4.227810319179846,
 4.22786829468405,
 4.227927920974303,
 4.228012500681451,
 4.228101867859131,
 4.228187655256651,
 4.228308704411872,
 4.228413168674158,
 4.228521988770672,
 4.228652066000313,
 4.22786829468405,
 4.227927920974303,
 4.228012500681451,
 4.228101867859131,
 4.228187655256651,
 4.228308704411872,
 4.228413168674158,
 4.228521988770672,
 4.228652066000313,
 4.228777921088388,
 4.215013250648896,
 4.191826261410885,
 4.1650134374366425,
 4.136942745050855,
 4.108206980059717,
 4.080007324549799,
 4.054729009661401,
 4.031174804881036,
 4.009030923983443,
 3.9885584630062167,
 3.969030792786378,
 3.9501859762958276,
 3.931585305523331,
 3.9134917984976467,
 3.89554012728947,
 3.878046332436983,
 3.861335595873233,
 3.8448572801811514,
 3.8288512446374217,
 3.81325329002614